## Day 7: Camel Card game (poker-esque)

Part 1: have `hands bids` lines, rank all these, then do: SUM hand_rank_i * bid_i

In [175]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

In [176]:
example_lines

['32T3K 765', 'T55J5 684', 'KK677 28', 'KTJJT 220', 'QQQJA 483']

Idea:
- Mappings from card to relative value such that we can use `sort`.
- First have arrange into types, then do `sort` on the mapped tuples of hands
  
Types are Five of a kind, Four of a kind, Full house (3 | 2), Three of a kind, Two pair, One pair, High Card

In [177]:
card_options = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]
card_mapping = {
    option: len(card_options)-i for i, option in enumerate(card_options)
}
mapping_to_card_mapping = {
    len(card_options)-i:option for i, option in enumerate(card_options)
}
card_mapping.__str__(), mapping_to_card_mapping.__str__()

("{'A': 13, 'K': 12, 'Q': 11, 'J': 10, 'T': 9, '9': 8, '8': 7, '7': 6, '6': 5, '5': 4, '4': 3, '3': 2, '2': 1}",
 "{13: 'A', 12: 'K', 11: 'Q', 10: 'J', 9: 'T', 8: '9', 7: '8', 6: '7', 5: '6', 4: '5', 3: '4', 2: '3', 1: '2'}")

In [178]:
FIVE = "Five of a kind"
FOUR = "Four of a kind"
FULL_HOUSE = "Full house"
THREE = "Three of a kind"
TWO = "Two pair"
ONE = "One pair"
HIGH = "High Card"
# basically using this like an enum but cba with using python enum today

hand_type_rankings = {
    option: 7-i for i, option in enumerate(
        [FIVE, FOUR, FULL_HOUSE, THREE, TWO, ONE, HIGH]
    )
}

# {'Five of a kind': 7, 'Four of a kind': 6, 'Full house': 5, 'Three of a kind': 4, 'Two pair': 3, 'One pair': 2, 'High Card': 1}

In [179]:
from collections import Counter

def classify_hand(hand_string: str) -> str:
    hand = [*hand_string]
    assert len(hand) == 5, "uh how many fingers does this man have?"
    hand_counter = Counter(hand)
    if 5 in hand_counter.values():
        return FIVE
    if 4 in hand_counter.values():
        return FOUR
    if 3 in hand_counter.values() and 2 in hand_counter.values():
        return FULL_HOUSE
    if 3 in hand_counter.values():
        return THREE
    if Counter(hand_counter.values())[2] == 2:
        return TWO
    if 2 in hand_counter.values():
        return ONE
    else:
        assert len(hand_counter.keys()) == 5
        return HIGH

In [180]:
def bucket_hands(hands_to_IDs: dict):
    fives, fours, fulls, threes, twos, ones, highs = [], [], [], [], [], [], []

    for hand in hands_to_IDs.keys():
        classification = classify_hand(hand)
        if classification == FIVE:
            fives.append(hand)
        elif classification == FOUR:
            fours.append(hand)
        elif classification == FULL_HOUSE:
            fulls.append(hand)
        elif classification == THREE:
            threes.append(hand)
        elif classification == TWO:
            twos.append(hand)
        elif classification == ONE:
            ones.append(hand)
        elif classification == HIGH:
            highs.append(hand)
        else:
            raise ValueError(f"What kind of hand is this? {hand}")
    
    return fives, fours, fulls, threes, twos, ones, highs 

In [181]:
def map_func_for_hand_to_val(card: str) -> int:
    return card_mapping[card]

In [182]:
def map_func_for_val_to_hand(card_val: int) -> str:
    return mapping_to_card_mapping[card_val]

In [183]:
v = "KKAAT"
(*v,)

('K', 'K', 'A', 'A', 'T')

In [184]:
def sort_hands(hands: list[str]) -> list[str]:
    hands = [(*hand_str,) for hand_str in hands]
    # converts hand str e.g. "KKAAT" to ("K", "K", "A", "A", "T")
    # Input of form 
    # [('K', 'K', '6', '7', '7'), ('K', 'T', 'J', 'J', 'T')]
    hands = [tuple(map(map_func_for_hand_to_val, hand)) for hand in hands]
    hands.sort(reverse=True)  # get to use nice inbuilt python sort that prioritise 1st elem, then 2nd etc. etc.
    # reverse = True means sort in descending order, so highest is at start of the list!
    hands = [tuple(map(map_func_for_val_to_hand, hand)) for hand in hands]
    hands = ["".join(hand_tuple) for hand_tuple in hands]
    return hands


In [185]:
b = ('K', 'K', '6', '7', '7')
"".join(b)

'KK677'

In [186]:
def part1(lines: list[str]):
    hands_to_IDs = {}
    IDs_to_bids = {}
    for i, line in enumerate(lines):
        hand, bid = line.split()
        hands_to_IDs[hand] = i
        IDs_to_bids[i] = int(bid)
    # for the example this looks like:
    # hands_to_IDs = {'32T3K': 0, 'T55J5': 1, 'KK677': 2, 'KTJJT': 3, 'QQQJA': 4}
    # IDs_to_bids = {0: 765, 1: 684, 2: 28, 3: 220, 4: 483}
    fives, fours, fulls, threes, twos, ones, highs = bucket_hands(hands_to_IDs)
    fives, fours, fulls, threes, twos, ones, highs = (
        sort_hands(fives),
        sort_hands(fours),
        sort_hands(fulls),
        sort_hands(threes),
        sort_hands(twos),
        sort_hands(ones),
        sort_hands(highs),
    )
    final_ordered_hands_list = fives + fours + fulls + threes + twos + ones + highs
    
    # Great we have an ordered list, now we can do our final answer
    running_total = 0
    for i, hand in enumerate(final_ordered_hands_list.__reversed__()):
        hand_ID = hands_to_IDs[hand]
        bid = IDs_to_bids[hand_ID]
        # Realising I could've probably mapped hands : bids, oh well!
        running_total += (i + 1) * bid

    return running_total

assert part1(example_lines) == 6440
part1(input_lines)

253313241

First try :)

**Part 2: messing with the bloody jokers**

So now J = Joker not Jack, so it's a wild card that becomes whatever is most useful!

But also now when ties are resolved, it's the least valuable...

In [187]:
# We need to change card values map

card_options = ["A", "K", "Q", "T", "9", "8", "7", "6", "5", "4", "3", "2", "J"]
card_mapping = {
    option: len(card_options)-i for i, option in enumerate(card_options)
}
mapping_to_card_mapping = {
    len(card_options)-i:option for i, option in enumerate(card_options)
}
card_mapping.__str__(), mapping_to_card_mapping.__str__()

("{'A': 13, 'K': 12, 'Q': 11, 'T': 10, '9': 9, '8': 8, '7': 7, '6': 6, '5': 5, '4': 4, '3': 3, '2': 2, 'J': 1}",
 "{13: 'A', 12: 'K', 11: 'Q', 10: 'T', 9: '9', 8: '8', 7: '7', 6: '6', 5: '5', 4: '4', 3: '3', 2: '2', 1: 'J'}")

In [188]:
# We also have to update (read: redefine lol) the classify_hand functions

def classify_hand(hand_string: str) -> str:
    hand = [*hand_string]
    assert len(hand) == 5, "uh how many fingers does this man have?"
    hand_counter = Counter(hand)
    if 5 in hand_counter.values():
        return FIVE
    if 4 in hand_counter.values():
        # If we had a joker as the fifth, then best bet is to change it to make a FIVE
        # JJJJK -> FIVE
        # KKKKJ -> FIVE
        # KKKKA -> Stay at FOUR
        if hand_counter["J"] in (1,4):
            return FIVE
        return FOUR
    if 3 in hand_counter.values() and 2 in hand_counter.values():
        # JJJKK -> FIVE
        # KKKJJ -> FIVE
        # KKKAA -> Stay at full
        if hand_counter["J"] in (2,3):
            return FIVE
        return FULL_HOUSE
    if 3 in hand_counter.values():
        # KKKJA -> FOUR
        # JJJKA -> FOUR
        # KKKA2 -> stays at THREE
        if hand_counter["J"] in (1,3):
            return FOUR
        return THREE
    if Counter(hand_counter.values())[2] == 2:
        # KKJJA -> FOUR
        # KKAAJ -> FULL
        # KKAA2 -> Stay at TWO
        if hand_counter["J"] == 2:
            return FOUR
        if hand_counter["J"] == 1:
            return FULL_HOUSE
        return TWO
    if 2 in hand_counter.values():
        # KKJA2 -> THREE
        # JJKA2 -> THREE
        # KKA23 -> Stay at ONE
        if hand_counter["J"] in (1,2):
            return THREE
        return ONE
    else:
        assert len(hand_counter.keys()) == 5
        # JKQA2 -> ONE
        # KQA23 -> stay at HIGH
        if hand_counter["J"] == 1:
            return ONE
        return HIGH

In [189]:
part2 = part1
assert part2(example_lines) == 5905
part2(input_lines)

253362743